In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# ---------------------------------------------------------
# Función para extraer tiempo (en segundos) de nombres: "+23 seg"
# ---------------------------------------------------------
def extraer_tiempo(nombre):
    nums = re.findall(r"\d+", nombre)
    if not nums:
        return None
    return float(nums[0])  # segundos

# ---------------------------------------------------------
# LEER ARCHIVO
# ---------------------------------------------------------
archivo = "Grillas_por_tiempo.xlsx"
xlsx = pd.ExcelFile(archivo)

tablas = []
tiempos = []

for hoja in xlsx.sheet_names:
    t = extraer_tiempo(hoja)
    if t is None:
        continue
    
    # Cargar con texto y números
    df = pd.read_excel(archivo, sheet_name=hoja, header=None)
    
    # Convertir todo a número (texto → NaN)
    df_num = df.apply(pd.to_numeric, errors="coerce")
    
    tablas.append(df_num.values.astype(float))
    tiempos.append(t)

# Ordenar por tiempo
tiempos, tablas = zip(*sorted(zip(tiempos, tablas)))
tablas = np.array(tablas)  # shape: (ntiempos, filas, columnas)

print("Se cargaron", len(tablas), "grillas.")
print("Tiempos (seg):", tiempos)

# ---------------------------------------------------------
# GRAFICAR EVOLUCIÓN TEMPORAL DE CADA CELDA
# ---------------------------------------------------------
num_t, filas, columnas = tablas.shape

for i in range(filas):
    for j in range(columnas):
        
        valores = tablas[:, i, j]  # valores a través del tiempo
        
        # Si todo es NaN → saltar
        if np.isnan(valores).all():
            continue
        
        # Cálculo barras de error (si hubiera replicados en el futuro)
        media = valores
        error = np.zeros_like(media)
        
        plt.figure(figsize=(6,4))
        plt.errorbar(tiempos, media, yerr=error, fmt="o-", capsize=3)
        
        plt.title(f"Evolución temporal — Celda ({i+1}, {j+1})")
        plt.xlabel("Tiempo (segundos)")
        plt.ylabel("Valor (numérico)")
        plt.grid(True)
        plt.tight_layout()
        plt.show()


ValueError: operands could not be broadcast together with shapes (33,13) (10,13) 